In [2]:
!pip install selenium

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [21]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [112]:
def get_jobs(keyword, num_jobs, verbose,path,slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.generalKeyword="' + keyword + '"%22data+scientist%22'
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            #driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
            driver.find_element_by_css_selector('[alt = "Close"]').click()
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:
            #print(job_button)

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            #print(len(jobs))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might
            #print("Button clicked")
            time.sleep(1)
            collected_successfully = False
            #print("Collected Successfully set")
            
            while not collected_successfully:
                #print("Job Length: ", len(jobs))
                i = int(len(jobs) + 1)
                try:
                    #company_name = driver.find_element_by_xpath('.//div[@class="jobHeader d-flex justify-content-between align-items-start"]/a/span').text
                    company_name = driver.find_element_by_xpath('.//li[@class="jl react-job-listing gdGrid selected"]/div[2]/div[1]/a/span').text
                    #company_name = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li[i]/div[2]/div[1]/a/span').text
                    #print(company_name)
                    #location = driver.find_element_by_xpath('.//div[@class="d-flex flex-wrap css-yytu5e e1rrn5ka1"]/span').text
                    location = driver.find_element_by_xpath('.//li[@class="jl react-job-listing gdGrid selected"]/div[2]/div[2]/span').text
                    #location = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li[1]/div[2]/div[2]/span').text
                    #print(location)
                    #job_title = driver.find_element_by_xpath('./div[contains(@class = "d-flex flex-column pl-sm css-nq3w9f")]/a/span').text
                    #job_title = driver.find_element_by_xpath('.//div[@class = "d-flex flex-column pl-sm css-nq3w9f"]/a/span').text
                    job_title = driver.find_element_by_xpath('.//li[@class="jl react-job-listing gdGrid selected"]/div[2]/a/span').text
                    #job_title = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li[1]/div[2]/a/span').text
                    #print(job_title)
                    #job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    #job_description = driver.find_element_by_xpath('//*[@id="JobDesc3788945477"]/div/div/p[1]/text()').text
                    #print(job_description)
                    collected_successfully = True
#                     print(collected_successfully)
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="css-56kyx5 css-16kxj2j e1wijj242"]').text
                #salary_estimate = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li[1]/div[2]/div[3]/div[1]/span/text()').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                #rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
                rating = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li[1]/div[1]/span/text()').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            #"Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [113]:
path = "C:/Python38/Git Repo/Salary-Project/chromedriver"
df = get_jobs('data scientist',15,False,path,15)

Progress: 0/15
Progress: 1/15
Progress: 2/15
Progress: 3/15
Progress: 4/15
Progress: 5/15
Progress: 6/15
Progress: 7/15
Progress: 8/15
Progress: 9/15
Progress: 10/15
Progress: 11/15
Progress: 12/15
Progress: 13/15
Progress: 14/15
Progress: 15/15


In [114]:
df.head()

,Job Title,Salary Estimate,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist Architect,-1,-1,Applied Information Sciences,Maryland,-1,501 to 1000 Employees,1982,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),-1
1,Principal Data Scientist,$104K - $170K (Glassdoor est.),-1,GEICO,"Chevy Chase, MD",-1,10000+ Employees,1936,Subsidiary or Business Segment,Insurance Carriers,Insurance,$10+ billion (USD),-1
2,Research Laboratory Scientist,-1,-1,"OmniSeq, Inc.","Buffalo, NY",-1,-1,-1,-1,-1,-1,-1,-1
3,SYSTEMS/DATA ENGINEER - ANALYTICS- INFORMATION...,$78K - $143K (Glassdoor est.),-1,St. Luke's Hospital,"Chesterfield, MO",-1,1001 to 5000 Employees,1885,Subsidiary or Business Segment,Health Care Services & Hospitals,Health Care,Less than $1 million (USD),-1
4,Certifying Scientist (mid-week start/weekends),-1,-1,Luxor Scientific,"Greenville, SC",-1,-1,-1,-1,-1,-1,-1,-1
